In [1]:
import sys
sys.path.append("/home/boldyrek/mysoft/te/te_reporting/")
import config_te_reporting as cfg
import os 
import importlib.util
from helper_functions import get_te_config_module, load_final_train_df,  get_te_constants_module
from helper_functions import start_spark_session, get_imputed_df ,get_te_load_pipeline_config_module
from col_stats import *

In [2]:
def check_every_aggregation_from_cfg( col ):
    """
    function aggregate_over_days does aggregations of multiple ctus to 
    on ctu in  base_enrich_data_pd.py
    it check cfg['rolling_agg_cols'], cfg['CUMMAX_COLS'], cfg['CUMMEDIAN_COLS']
    cfg['ID_COL'] it is a lag features that adds _lag_ to column name
    DiFF [x+'_diff' for x in df_diff.columns]
     Creates rate columns cols = [x+'_rate' for x in rate_cols]
     create cyclical feature 'df[feat+'_sin', df[feat+'_cos']
     lag features

    """
    te_constants_module = get_te_constants_module()
    col_aggregations = ''
    for aggregation in cfg.AGGREGATIONS:
        if col  in getattr(te_constants_module, aggregation):
            col_aggregations += aggregation + ',' 
    
    return col_aggregations

def get_cols_aggregations_df( columns):
    
    col_and_its_aggregation = []
    for col in columns:
        col_aggregations = check_every_aggregation_from_cfg( col )
        col_and_its_aggregation.append((col,col_aggregations))       
    col_and_its_aggregation = spark.createDataFrame(col_and_its_aggregation, ["column_name_agg", "aggregation"])
    return col_and_its_aggregation
 

In [12]:

spark = start_spark_session()
# STEP 2
te_constants_module =  get_te_constants_module()
final_train_df = get_imputed_df(cfg.IMPUTATION_TRAIN_PATH, cfg.IMPUTATION_PREDICT_PATH)
columns_final_train = final_train_df.columns
col_and_its_aggregation = get_cols_aggregations_df( columns_final_train )

In [13]:
!ls /home/boldyrek/mysoft/te/data_plus_enrichment_funcs/

enriched_train_2020_08_17_1.csv
enriched_train_2020_08_17_1.feather
enriched_train_2020_08_17_no_outliers_1.feather
final_predict_2020_08_17.pkl
final_train_2020_08_17_1.pkl
final_train_2020_08_17_2.pkl
final_train_2020_08_17_3.pkl
final_train_2020_08_17_full.pkl
final_train_test_out_of_sample_2020_08_17.pkl
general_preprocessing_2020_08_17_1.csv
general_preprocessing_2020_08_17_1.feather
imputed_predict_2020_08_17_1.csv
imputed_train_2020_08_17_1.csv
imputed_train_2020_08_17_1.feather
model_LGBM_2020_08_17.pkl
pred_scores_LGBM_2020_08_17.pkl
pred_scores_LGBM_train_2020_08_17.pkl
preprocessing_2020_08_17_1.csv
preprocessing_2020_08_17_1.feather
sampled_2020_08_17_1.pkl
sampled_2020_08_17_2.pkl
sampled_2020_08_17_3.pkl
sampled_2020_08_17_full.pkl
split_pred_2020_08_17_1.pkl
split_train_2020_08_17_1.pkl
split_validate_2020_08_17_1.pkl
Te_logfile_2020_08_17.log


In [14]:
# STEP 4
imputed_df = get_imputed_df(cfg.IMPUTATION_TRAIN_PATH, cfg.IMPUTATION_PREDICT_PATH)
final_train_descriptive_stats = get_df_with_descriptive_stats_for_columns ( spark ,
                                                                           imputed_df)

In [15]:
# STEP 5
final_train_descriptive_stats.columns
col_and_its_aggregation.columns

['column_name_agg', 'aggregation']

In [17]:
imputed_df.columns

['party_id',
 'CTU',
 'level_0',
 'index',
 'Unnamed: 0',
 'event_date',
 'cai_ins_grs_vmc',
 'cai_ins_grs_mrc',
 'cai_ins_grs_erc',
 'cai_ins_grs_evmc',
 'cai_ins_grs_vuc',
 'cai_ins_grs_evnt_1',
 'cai_ins_grs_evnt_2',
 'cai_ins_grs_evnt_3',
 'cai_ins_grs_rand',
 'cai_factor_age',
 'cai_factor_1',
 'cai_factor_2',
 'event_id',
 'expanding_cai_ins_grs_vmc',
 'expanding_cai_ins_grs_mrc',
 'expanding_cai_ins_grs_erc',
 'expanding_cai_ins_grs_evmc',
 'expanding_cai_ins_grs_vuc',
 'expanding_cai_ins_grs_evnt_1',
 'expanding_cai_ins_grs_evnt_2',
 'expanding_cai_ins_grs_evnt_3',
 'REF_EVENT',
 'td_start',
 'year',
 'month',
 'yr_month',
 'event_inbound_interactions',
 'event_outbound_interactions',
 'event_positive_interactions',
 'event_negative_interactions',
 'td_last_cai_ins_grs_vmc',
 'td_last_cai_ins_grs_mrc',
 'td_last_cai_ins_grs_erc',
 'td_last_cai_ins_grs_evmc',
 'td_last_cai_ins_grs_vuc',
 'td_last_cai_ins_grs_evnt_1',
 'td_last_cai_ins_grs_evnt_2',
 'td_last_cai_ins_grs_evnt_3',


In [16]:
columns

NameError: name 'columns' is not defined

In [ ]:
# STEP 3
"""
All the affixes for this step are derived from base_enrich_data_pd.py and 
can be manually added to config.py
We use columns names to see if they have enrichment affixes
Pseudocode:
1. Take one column
2. pass col to get enrichment type
    a. in enrichment type loop through all the affixes and if the affix is inside a colum
    cut the affix taking last part of it _mean, an remove underscore
    if there is a _lag_5 then remove lag_5 for example
    
"""

def get_enrichment_type( col ):
    import re
    for affix in cfg.ENRICHMENT_AFFIXES:
        if affix not in col:
            #print(f"{affix} not in {col}" )
            continue
        elif '_lag_' in col:
                return 'lag'
        else:
           transformation = affix.split('_')
           return transformation[-1]
        return None
    
col_and_enrichment = []    
for col in columns:
    enrichment_type = get_enrichment_type( col ) 
    if enrichment_type != None:
        print(col)
        print(enrichment_type)
        col_and_enrichment.append((col,enrichment_type))
    col_and_enrichment.append((col,'None'))


In [ ]:
col_and_enrichment

In [ ]:
columns = [
'cai_ins_grs_cntrb_m_vol_unmtch_amt_rolling_std',
'cai_ins_grs_cntrb_m_vol_unmtch_amt_rolling_trend',
'cai_ins_grs_cntrb_m_vol_unmtch_amt_diff',
'year_sin',
'mont_cos',
'event_rate',
'evet1_lag_1',
'evetn2_diff',
'price_per_unit',
'payment_fail_ratio',
'event3_rolling_trend']

In [ ]:
for affix in cfg.ENRICHMENT_AFFIXES:
    print(f"{affix}")

In [ ]:
col = 'my_rolling_trend'

In [ ]:
affix

In [ ]:
"""
Step1 see if affix inside the column
remove leading underscore
"""
import re
match = re.search(affix, col)
if match != None: print(match[0])
    re.findall(r'_\w*_(\w*)' , affix)

In [ ]:
affix

In [ ]:
cfg.ENRICHMENT_AFFIXES

In [ ]:
# getting last part of an affix
re.search(r'_\b.*\b' , affix)

In [ ]:
affix ="_diff"
affix = '_rolling_trend'
re.findall(r'^[_]+' , affix)

In [ ]:
affix.split('_')

In [6]:
spark = start_spark_session()

In [7]:
import pandas as pd